#### Link To The Video:
#### https://www.youtube.com/watch?v=hvLFD4AZzCw&list=PLyMom0n-MBroupZiLfVSZqK5asX8KfoHL&index=3

### Initialization

In [1]:
# Import necessary libraries for building and training a neural network using the PyTorch framework
import copy                       # For creating deep and shallow copies of Python objects
import torch.nn as nn             # Neural network module in PyTorch
import torch.nn.functional as F   # Contains various functions for building neural networks
import torch                      # Provides functions and classes for working with tensors and neural networks
import PIL                        # Provides functions for working with images in Python
import numpy as np                # Provides functions for working with numerical arrays in Python

# Import the MNIST dataset
from torch.utils.data import DataLoader, random_split  # For loading and iterating over datasets in PyTorch, and splitting a dataset into training and validation subsets
from torchvision.datasets import MNIST                 # PyTorch dataset class that provides access to the MNIST dataset


In [2]:
# Load the MNIST dataset and split it into training and validation subsets
dataset = MNIST(root='../../data/', download=True)
train_ds_, validation_ds_ = random_split(dataset, [50000, 10000])

# Convert the image and label data to PyTorch tensors and store them in separate lists for the training and validation subsets
train_ds = [(torch.tensor(np.asarray(image_data)).to(torch.float32), (torch.tensor(label)).to(torch.int64)) for image_data, label in train_ds_]
validation_ds = [(torch.tensor(np.asarray(image_data)).to(torch.float32), (torch.tensor(label)).to(torch.int64)) for image_data, label in validation_ds_]

# Create data loaders for iterating over the training and validation subsets in batches
batch_size = 128
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
validation_loader = DataLoader(validation_ds, batch_size)


### Definitions & Model

In [3]:
# Define a function to calculate accuracy
def accuracy(outputs, labels):

    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [4]:
class MnistModel(nn.Module):

    VALIDATION_STEP_OUTPUT_TEMPLATE = {"loss": None, "accuracy": None}

    def __init__(self, in_features, out_features):

        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.loss_fn = F.cross_entropy

    def forward(self, xb: torch.Tensor):
        xb = xb.reshape(-1, 784)  # We request the other dimension to be calculated by pytorch
        return self.linear(xb)
    
    def training_step(self, batch: list[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        
        images, labels = batch
        outputs = self(images)
        return self.loss_fn(outputs, labels)
    
    def validation_step(self, batch: list[torch.Tensor, torch.Tensor]) -> dict:

        images, labels = batch
        outputs = self(images)
        t_loss = self.loss_fn(outputs, labels)
        t_accuracy = accuracy(outputs, labels)
        
        output_template = copy.copy(MnistModel.VALIDATION_STEP_OUTPUT_TEMPLATE)
        output_template["loss"] = t_loss
        output_template["accuracy"] = t_accuracy
        return output_template
    
    def validation_epoch_end(self, validation_step_outputs: dict):

        batch_losses = [result["loss"] for result in validation_step_outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        batch_accuracies = [result["accuracy"] for result in validation_step_outputs]
        epoch_accuracy = torch.stack(batch_accuracies).mean()  # Combine accuracies
        
        output_template = copy.copy(MnistModel.VALIDATION_STEP_OUTPUT_TEMPLATE)
        output_template["loss"] = epoch_loss.item()  # Get value using ".item()"
        output_template["accuracy"] = epoch_accuracy.item()
        return output_template
    
    def print_result(self, epoch, result):

        print(f"Epoch [{epoch+1}], loss: {result['loss']:.4f}, accuracy: {result['accuracy']:.4f}")


### Training The Model

In [5]:
# Define a function to calculate accuracy
def evaluate(model, validation_loader):

    outputs = [model.validation_step(batch) for batch in validation_loader]
    return model.validation_epoch_end(outputs)

In [6]:
def fit(epochs, lr, model, train_loader, validation_loader, optimizer_function=torch.optim.SGD):

    optimizer = optimizer_function(model.parameters(), lr)
    history = []  # To record what happens during the training.

    for epoch in range(epochs):

        # Training phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()       # Parameter update.
            optimizer.zero_grad()  # Reset gradients.

        # Validation phase
        result = evaluate(model, validation_loader)
        model.print_result(epoch, result)
        history.append(result)

    return history

In [7]:
model = MnistModel(28*28, 10)

In [8]:
evaluate(model, validation_loader)

{'loss': 95.21365356445312, 'accuracy': 0.0768394023180008}

In [44]:
fit(50, .00001, model, train_loader, validation_loader);

Epoch [1], loss: 0.7240, accuracy: 0.8909
Epoch [2], loss: 0.7189, accuracy: 0.8915
Epoch [3], loss: 0.7133, accuracy: 0.8912
Epoch [4], loss: 0.7183, accuracy: 0.8923
Epoch [5], loss: 0.7175, accuracy: 0.8914
Epoch [6], loss: 0.7316, accuracy: 0.8876
Epoch [7], loss: 0.7123, accuracy: 0.8896
Epoch [8], loss: 0.7044, accuracy: 0.8923
Epoch [9], loss: 0.6961, accuracy: 0.8929
Epoch [10], loss: 0.6995, accuracy: 0.8940
Epoch [11], loss: 0.6968, accuracy: 0.8925
Epoch [12], loss: 0.7076, accuracy: 0.8892
Epoch [13], loss: 0.6846, accuracy: 0.8927
Epoch [14], loss: 0.7056, accuracy: 0.8886
Epoch [15], loss: 0.6910, accuracy: 0.8920
Epoch [16], loss: 0.6880, accuracy: 0.8920
Epoch [17], loss: 0.6843, accuracy: 0.8927
Epoch [18], loss: 0.6787, accuracy: 0.8928
Epoch [19], loss: 0.6736, accuracy: 0.8935
Epoch [20], loss: 0.6847, accuracy: 0.8907
Epoch [21], loss: 0.6672, accuracy: 0.8944
Epoch [22], loss: 0.6668, accuracy: 0.8924
Epoch [23], loss: 0.6873, accuracy: 0.8894
Epoch [24], loss: 0.

In [10]:
train_ds__, validation_ds__ = random_split(dataset, [50000, 10000])

In [45]:
image = PIL.Image.open("/home/lyonbach/Documents/eight_2ie.jpg")
image_as_array = np.asarray(image)[:,:, 1].reshape(28, 28, 1)
image_as_tensor = torch.Tensor(image_as_array)
results = model(image_as_tensor).tolist()[0]
print(results.index(max(results)))
image.show()

3


In [38]:
image, label = train_ds__[280]

image_as_array = np.asarray(image)
image_as_tensor = torch.Tensor(image_as_array)
results = model(image_as_tensor).tolist()[0]


print(results.index(max(results)))

image.show()

9
